# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet','averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wythe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wythe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wythe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# import libraries
import sys
import sqlite3
import os
from sqlalchemy import create_engine
import pandas as pd
import numpy as np


In [3]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score, make_scorer
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [4]:
# load data from database

engine = create_engine('sqlite:///DisasterResponse.db') 
df = pd.read_sql("select * from df", engine)
X = df.message.values
y = df[df.columns[4:]]
category_names = y.columns    


In [5]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier(n_jobs=-1))),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [9]:
# split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.2)

In [10]:
# train classifier
pipeline.fit(X_train, y_train)   
   

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001C055A12288>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
# predict
y_pred = pipeline.predict(X_test)


In [12]:
print(classification_report(y_test, y_pred, target_names = category_names))


                        precision    recall  f1-score   support

               related       0.80      0.97      0.88     16072
               request       0.91      0.36      0.52      3609
                 offer       0.00      0.00      0.00        96
           aid_related       0.79      0.57      0.66      8690
          medical_help       0.57      0.03      0.06      1647
      medical_products       0.71      0.04      0.07      1046
     search_and_rescue       0.67      0.00      0.01       580
              security       0.00      0.00      0.00       363
              military       0.63      0.03      0.05       663
           child_alone       0.00      0.00      0.00         0
                 water       0.94      0.15      0.26      1333
                  food       0.89      0.31      0.46      2351
               shelter       0.90      0.18      0.31      1844
              clothing       0.73      0.02      0.05       333
                 money       1.00      

C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_cla

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = parameters = {
        'clf__estimator__n_estimators': [50, 100],
        'clf__estimator__min_samples_leaf': [5, 10]
}
    
cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2)  

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
cv.fit(X_train, y_train)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:  1.9min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001C055A12288>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))]),
             n_jobs=4,
             param_grid={'clf__estimator__min_samples_leaf': [5, 10],
                         'clf__estimator__n_estimators': [50, 100]},
             verbose=2)

In [15]:
y_pred_improved = cv.predict(X_test)

In [16]:
print(classification_report(y_test, y_pred_improved, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87     16072
               request       0.98      0.01      0.02      3609
                 offer       0.00      0.00      0.00        96
           aid_related       0.82      0.45      0.58      8690
          medical_help       0.00      0.00      0.00      1647
      medical_products       0.00      0.00      0.00      1046
     search_and_rescue       0.00      0.00      0.00       580
              security       0.00      0.00      0.00       363
              military       0.00      0.00      0.00       663
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00      1333
                  food       0.00      0.00      0.00      2351
               shelter       0.00      0.00      0.00      1844
              clothing       0.00      0.00      0.00       333
                 money       0.00      

C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_cla

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
best_model = cv.best_params_

In [18]:
best_model

{'clf__estimator__min_samples_leaf': 10, 'clf__estimator__n_estimators': 100}

In [19]:
pipeline_1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100,min_samples_leaf=10,
                                                             n_jobs=-1)))
    ])

In [20]:
pipeline_1.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001C055A12288>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_leaf=10,
                                                                        n_jobs=-1)))])

In [21]:
y_pred = pipeline_1.predict(X_test)

In [22]:
print(classification_report(y_test, y_pred,target_names=category_names))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87     16072
               request       1.00      0.00      0.01      3609
                 offer       0.00      0.00      0.00        96
           aid_related       0.83      0.42      0.56      8690
          medical_help       0.00      0.00      0.00      1647
      medical_products       0.00      0.00      0.00      1046
     search_and_rescue       0.00      0.00      0.00       580
              security       0.00      0.00      0.00       363
              military       0.00      0.00      0.00       663
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00      1333
                  food       0.00      0.00      0.00      2351
               shelter       0.00      0.00      0.00      1844
              clothing       0.00      0.00      0.00       333
                 money       0.00      

C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wythe\Anaconda3\lib\site-packages\sklearn\metrics\_cla

### 9. Export your model as a pickle file

In [25]:
with open('model.pkl', 'wb') as model_file:
  pickle.dump(pipeline_1, model_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.